In [15]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import r2_score,accuracy_score
from sklearn.model_selection import train_test_split,RandomizedSearchCV,StratifiedKFold
from sklearn.ensemble import BaggingRegressor,RandomForestClassifier,StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder

In [5]:
img=pd.read_csv("Image_Segmention.csv")
lbl=LabelEncoder()
img['Class']=lbl.fit_transform(img['Class'])
img.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,0,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,0,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,0,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,0,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,0,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [6]:
X=img.drop("Class",axis=1)
y=img['Class']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=23)

In [8]:
lda=LinearDiscriminantAnalysis()
svm=SVC(kernel='linear',random_state=23,probability=True)
gnb=GaussianNB()
lr=LogisticRegression(random_state=23)
xgboost=XGBClassifier()

In [12]:
stack=StackingClassifier([('LR',lr),('SVM',svm),('GNB',gnb),('LDA',lda)],final_estimator=xgboost)

In [13]:
stack.get_params()

{'cv': None,
 'estimators': [('LR', LogisticRegression(random_state=23)),
  ('SVM', SVC(kernel='linear', probability=True, random_state=23)),
  ('GNB', GaussianNB()),
  ('LDA', LinearDiscriminantAnalysis())],
 'final_estimator__objective': 'binary:logistic',
 'final_estimator__base_score': None,
 'final_estimator__booster': None,
 'final_estimator__callbacks': None,
 'final_estimator__colsample_bylevel': None,
 'final_estimator__colsample_bynode': None,
 'final_estimator__colsample_bytree': None,
 'final_estimator__device': None,
 'final_estimator__early_stopping_rounds': None,
 'final_estimator__enable_categorical': False,
 'final_estimator__eval_metric': None,
 'final_estimator__feature_types': None,
 'final_estimator__gamma': None,
 'final_estimator__grow_policy': None,
 'final_estimator__importance_type': None,
 'final_estimator__interaction_constraints': None,
 'final_estimator__learning_rate': None,
 'final_estimator__max_bin': None,
 'final_estimator__max_cat_threshold': None,
 

In [14]:
params={'GNB__var_smoothing':np.linspace(0.0001,0.01,5),'LR__penalty':['l1','l2',None],'final_estimator__n_estimators':[10,15,50],
        'SVM__C':np.linspace(0.001,3,5),'final_estimator__learning_rate':[0.1,0.2],
       'final_estimator__max_depth':[2,3,4,5,6,None],'LDA__solver':['svd', 'lsqr', 'eigen']}

In [17]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
gcv_stack=RandomizedSearchCV(stack,param_distributions=params,cv=kfold,scoring='neg_log_loss',verbose=3)

In [18]:
gcv_stack.fit(X,y)  

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END GNB__var_smoothing=0.002575, LDA__solver=eigen, LR__penalty=l2, SVM__C=1.5005, final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=10;, score=nan total time=   0.5s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5] END GNB__var_smoothing=0.002575, LDA__solver=eigen, LR__penalty=l2, SVM__C=1.5005, final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=10;, score=nan total time=   0.3s


...

[CV 4/5] END GNB__var_smoothing=0.0001, LDA__solver=eigen, LR__penalty=l2, SVM__C=1.5005, final_estimator__learning_rate=0.2, final_estimator__max_depth=4, final_estimator__n_estimators=10;, score=nan total time=   0.3s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5] END GNB__var_smoothing=0.0001, LDA__solver=eigen, LR__penalty=l2, SVM__C=1.5005, final_estimator__learning_rate=0.2, final_estimator__max_depth=4, final_estimator__n_estimators=10;, score=nan total time=   0.3s
[CV 1/5] END GNB__var_smoothing=0.005050000000000001, LDA__solver=lsqr, LR__penalty=l1, SVM__C=0.75075, final_estimator__learning_rate=0.1, final_estimator__max_depth=5, final_estimator__n_estimators=15;, score=nan total time=   0.0s
[CV 2/5] END GNB__var_smoothing=0.005050000000000001, LDA__solver=lsqr, LR__penalty=l1, SVM__C=0.75075, final_estimator__learning_rate=0.1, final_estimator__max_depth=5, final_estimator__n_estimators=15;, score=nan total time=   0.0s
[CV 3/5] END GNB__var_smoothing=0.005050000000000001, LDA__solver=lsqr, LR__penalty=l1, SVM__C=0.75075, final_estimator__learning_rate=0.1, final_estimator__max_depth=5, final_estimator__n_estimators=15;, score=nan total time=   0.0s
[CV 4/5] END GNB__var_smoothing=0.005050000000000001, LDA__solver=lsqr, LR__pe

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
35 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_stacking.py", line 658, in fit
    return super().fit(X, y_encoded, sample_weight)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1152, in

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
                   estimator=StackingClassifier(estimators=[('LR',
                                                             LogisticRegression(random_state=23)),
                                                            ('SVM',
                                                             SVC(kernel='linear',
                                                                 probability=True,
                                                                 random_state=23)),
                                                            ('GNB',
                                                             GaussianNB()),
                                                            ('LDA',
                                                             LinearDiscriminantAnalysis())],
                                                final_estimator=XGBClassifier(base_score=None,
                                                                              booster=None,
                                                                              callbac...
                   param_distributions={'GNB__var_smoothing': array([0.0001  , 0.002575, 0.00505 , 0.007525, 0.01    ]),
                                        'LDA__solver': ['svd', 'lsqr', 'eigen'],
                                        'LR__penalty': ['l1', 'l2', None],
                                        'SVM__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00]),
                                        'final_estimator__learning_rate': [0.1,
                                                                           0.2],
                                        'final_estimator__max_depth': [2, 3, 4,
                                                                       5, 6,
                                                                       None],
                                        'final_estimator__n_estimators': [10,
                                                                          15,
                                                                          50]},
                   scoring='neg_log_loss', verbose=3)

In [19]:
print(gcv_stack.best_params_)
print(gcv_stack.best_score_)

{'final_estimator__n_estimators': 10, 'final_estimator__max_depth': 2, 'final_estimator__learning_rate': 0.2, 'SVM__C': 3.0, 'LR__penalty': 'l2', 'LDA__solver': 'lsqr', 'GNB__var_smoothing': 0.005050000000000001}
-0.5271098554965882
